In [8]:
from timeit import default_timer as timer
# We import all our dependencies.
import argparse
import os
import numpy as np 
import tifffile as tif
import csv
import psutil
from scipy import ndimage, spatial, stats
from tqdm import tqdm
import operator

In [9]:
# load image and make sure it's binary
img_path = '/Users/aichah/Documents/TLI_analysis/input_data/Rigid_GFP_1_220209_brain2.2_LP40_P36_neuron1.tif'
with tif.TiffFile(img_path) as tiff:
    img = tiff.asarray()
img[img!=0] = 1

In [10]:
def segment_3D(image, neu_no=1, max_neu_no=1, min_size=5000):
    s = ndimage.generate_binary_structure(len(image.shape), len(image.shape))
    labeled_array, num_labels = ndimage.label(image, structure=s)
    labels = np.unique(labeled_array)
    labels = labels[labels!=0]
    neu_sizes = {}
    for l in labels:
        neu_sizes[l] = (labeled_array == l).sum()/(labeled_array == l).max()
        # print((labeled_array == l).sum(), neu_sizes[l])
    avg_size = np.mean(list(neu_sizes.values()))
    # print('average, min and max segments sizes', avg_size, np.min(list(neu_sizes.values())), np.max(list(neu_sizes.values())))
    if min_size != 0:
        for ind, l in enumerate(labels):
            if neu_sizes[l] < min_size:
                # print(neu_sizes[l])
                labels[ind] = 0
        labels = labels[labels!=0]
    if neu_no != 0 and num_labels > neu_no:
        for ind, l in enumerate(labels):
            if neu_sizes[l] < avg_size:
                # print(neu_sizes[l])
                labels[ind] = 0
        labels = labels[labels!=0]
        # print('segments after first filtering', len(labels))
    if max_neu_no != 0 and len(labels) > max_neu_no:
        sorted_sizes = sorted(neu_sizes.items(), key=operator.itemgetter(1), reverse=True)
        sorted_sizes = sorted_sizes[0:max_neu_no]
        labels = [[l][0][0] for l in sorted_sizes]
        # print('# segments after second filtering', len(labels))
    # print('segments after first filtering', len(labels))
    neurons = {}
    for ind, l in enumerate(labels):
        labels[ind] = ind+1
        neuron = labeled_array.copy()
        neuron[neuron != l] = 0
        neuron[neuron == l] = ind+1
        neuron = neuron.astype('uint16')
        # print('values and size of neuron:', neuron.min(), neuron.max(), neuron.sum()/(ind+1))
        if neuron.sum() != 0 and neuron.sum() < np.prod(np.array(neuron.shape)):
            neurons[ind+1] = neuron
        else:
            print('this segment was removed because its empty')            
    return neurons


In [12]:
img1 = np.zeros_like(img)
for t in tqdm(range(len(img))):
    img1[t] = segment_3D(img[t], neu_no=1, max_neu_no=1, min_size=5000)[1]

 40%|███▉      | 35/88 [1:29:27<1:56:22, 131.74s/it]

In [ ]:
tif.imsave('../input_data/220209_neuron1_gfp_clean.tif', img1, imagej=True)